In [1]:
!git clone https://github.com/OFA-Sys/OFA.git

Cloning into 'OFA'...
remote: Enumerating objects: 6680, done.
remote: Counting objects: 100% (1362/1362), done.
remote: Compressing objects: 100% (383/383), done.
remote: Total 6680 (delta 1051), reused 991 (delta 978), pack-reused 5318
Receiving objects: 100% (6680/6680), 122.81 MiB | 35.66 MiB/s, done.
Resolving deltas: 100% (2845/2845), done.


In [2]:
!mkdir -p OFA/checkpoints/
!wget https://ofa-silicon.oss-us-west-1.aliyuncs.com/checkpoints/refcocog_large_best.pt
!mv refcocog_large_best.pt OFA/checkpoints/refcocog.pt

--2023-06-09 13:29:23--  https://ofa-silicon.oss-us-west-1.aliyuncs.com/checkpoints/refcocog_large_best.pt
Resolving ofa-silicon.oss-us-west-1.aliyuncs.com (ofa-silicon.oss-us-west-1.aliyuncs.com)... 47.88.111.45
Connecting to ofa-silicon.oss-us-west-1.aliyuncs.com (ofa-silicon.oss-us-west-1.aliyuncs.com)|47.88.111.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6230619243 (5.8G) [application/octet-stream]
Saving to: ‘refcocog_large_best.pt’

refcocog_large_best 100%[===================>]   5.80G  24.4MB/s    in 4m 6s   

2023-06-09 13:33:30 (24.1 MB/s) - ‘refcocog_large_best.pt’ saved [6230619243/6230619243]



In [3]:
!git clone https://github.com/pytorch/fairseq.git -b v0.12.0

Cloning into 'fairseq'...
remote: Enumerating objects: 34724, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 34724 (delta 92), reused 141 (delta 68), pack-reused 34543
Receiving objects: 100% (34724/34724), 25.00 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (25193/25193), done.
Note: switching to '6795311bfeb9d39fe11a62803184b81acb66509e'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [4]:
cd fairseq

/content/fairseq


In [5]:
!pip -q install --use-feature=no-binary-enable-wheel-cache ./

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.7/273.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
cd ../OFA

/content/OFA


In [7]:
%%capture
!sed '1d' requirements.txt | xargs -I {} pip install {}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=bb9e5e52d7913d2a56e326bc97c3b2dc8eab4d99eda48c58ffc62ffa885133d0
  Stored in directory: /root/.cache/pip/wheels/92/8e/16/c1e4d4d65685d71085e4e27b44d6ed880b0559474c9ee4ff66
Successfully built ftfy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [9]:
!pip -q install wget

  Preparing metadata (setup.py) ... done


In [11]:
#!g1.1
import torch
import numpy as np
from fairseq import utils, tasks
from fairseq import checkpoint_utils
from utils.eval_utils import eval_step
from tasks.mm_tasks.refcoco import RefcocoTask
from models.ofa import OFAModel
from PIL import Image

# Register refcoco task
tasks.register_task('refcoco', RefcocoTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [12]:
#!g1.1
# Load pretrained ckpt & config
overrides={"bpe_dir":"utils/BPE"}
models, cfg, task = checkpoint_utils.load_model_ensemble_and_task(
        utils.split_paths('checkpoints/refcocog.pt'),
        arg_overrides=overrides
    )

cfg.common.seed = 7

# Fix seed for stochastic decoding
if cfg.common.seed is not None and not cfg.generation.no_seed_provided:
    np.random.seed(cfg.common.seed)
    utils.set_torch_seed(cfg.common.seed)

# Move models to GPU
for model in models:
    model.eval()
    if use_fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
#!g1.1
# Image transform
from torchvision import transforms
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((cfg.task.patch_image_size, cfg.task.patch_image_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Text preprocess
bos_item = torch.LongTensor([task.src_dict.bos()])
eos_item = torch.LongTensor([task.src_dict.eos()])
pad_idx = task.src_dict.pad()
def encode_text(text, length=None, append_bos=False, append_eos=False):
    s = task.tgt_dict.encode_line(
        line=task.bpe.encode(text.lower()),
        add_if_not_exist=False,
        append_eos=False
    ).long()
    if length is not None:
        s = s[:length]
    if append_bos:
        s = torch.cat([bos_item, s])
    if append_eos:
        s = torch.cat([s, eos_item])
    return s

# Construct input for refcoco task
patch_image_size = cfg.task.patch_image_size
def construct_sample(image: Image, text: str):
    w, h = image.size
    w_resize_ratio = torch.tensor(patch_image_size / w).unsqueeze(0)
    h_resize_ratio = torch.tensor(patch_image_size / h).unsqueeze(0)
    patch_image = patch_resize_transform(image).unsqueeze(0)
    patch_mask = torch.tensor([True])
    src_text = encode_text(' which region does the text " {} " describe?'.format(text), append_bos=True, append_eos=True).unsqueeze(0)
    src_length = torch.LongTensor([s.ne(pad_idx).long().sum() for s in src_text])
    sample = {
        "id":np.array(['42']),
        "net_input": {
            "src_tokens": src_text,
            "src_lengths": src_length,
            "patch_images": patch_image,
            "patch_masks": patch_mask,
        },
        "w_resize_ratios": w_resize_ratio,
        "h_resize_ratios": h_resize_ratio,
        "region_coords": torch.randn(1, 4)
    }
    return sample
  
# Function to turn FP32 to FP16
def apply_half(t):
    if t.dtype is torch.float32:
        return t.to(dtype=torch.half)
    return t

In [14]:
#!g1.1
def show_box(box, ax, color):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor=color, facecolor=(0,0,0,0), lw=2))  

In [15]:
#!g1.1
def get_iou(bb1, bb2):
    # Taken from https://stackoverflow.com/a/42874377
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [16]:
#!g1.1
import pandas as pd
import os
import cv2
import numpy
import matplotlib.pyplot as plt
from tqdm import tqdm

from joblib import Parallel, delayed
import wget

In [17]:
vqa_answers = pd.read_csv('../vqa_answers.csv')

In [18]:
os.mkdir('../imgs/')
img_paths = Parallel(
    n_jobs=100)(delayed(wget.download)(img_url, out='../imgs') for img_url in tqdm(vqa_answers.image)
)

100%|██████████| 600/600 [00:45<00:00, 13.09it/s]


In [19]:
bounding_boxes = []
sum_iou = 0
n = 0
progress = tqdm(vqa_answers.iterrows(), total=len(vqa_answers))
for _, row in progress:
    image_path = row['image'].split('/')[-1]
    image = Image.open(os.path.join('../imgs', image_path))
    text = row['answer']

    # Construct input sample & preprocess for GPU if cuda available
    sample = construct_sample(image, text)
    sample = utils.move_to_cuda(sample) if use_cuda else sample
    sample = utils.apply_to_sample(apply_half, sample) if use_fp16 else sample

    # Run eval step for open-domain VQA
    with torch.no_grad():
        result, scores = eval_step(task, generator, models, sample)
            
    pred_box = [int(x) for x in result[0]["box"]]
    bounding_boxes.append(pred_box)
    gt_box = [row[key] for key in ('left', 'top', 'right', 'bottom')]
    iou = get_iou(
        {'x1': gt_box[0], 'y1': gt_box[1], 'x2': gt_box[2], 'y2': gt_box[3]},
        {'x1': pred_box[0], 'y1': pred_box[1], 'x2': pred_box[2], 'y2': pred_box[3]}
    )
    sum_iou += iou
    n += 1
    avg_iou = sum_iou / n * 100
    progress.set_description(f'IoU: {round(avg_iou, 2)}')

IoU: 37.57: 100%|██████████| 600/600 [05:28<00:00,  1.83it/s]


In [20]:
vqa_answers['pred_left'] = [x[0] for x in bounding_boxes]
vqa_answers['pred_top'] = [x[1] for x in bounding_boxes]
vqa_answers['pred_right'] = [x[2] for x in bounding_boxes]
vqa_answers['pred_bottom'] = [x[3] for x in bounding_boxes]
image_grounding = vqa_answers

In [21]:
image_grounding.to_csv('../ofa_image_grounding.csv', index=False)

# SAM re-annotation

In [22]:
cd ..

/content


In [23]:
!wget "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"

--2023-06-09 13:44:23--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.160.108, 99.84.160.80, 99.84.160.61, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.160.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   203MB/s    in 18s     

2023-06-09 13:44:41 (136 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [24]:
!pip -q install git+https://github.com/facebookresearch/segment-anything.git

  Preparing metadata (setup.py) ... done


In [25]:
!pip -q install opencv-python pycocotools matplotlib onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB/s eta 0:00:00


In [26]:
from segment_anything import SamPredictor, sam_model_registry
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import requests
from io import BytesIO
from tqdm.notebook import tqdm
import matplotlib.patches as mpatches

In [27]:
def get_bounding_box(mask):
    """
    Get the bounding box of a segmentation mask in the form of a NumPy bool array.
    
    Args:
        mask (NumPy array): The segmentation mask as a NumPy bool array.
        
    Returns:
        A NumPy array of the bounding box in the format [left, top, right, bottom].
    """
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    left, right = np.where(cols)[0][[0, -1]]
    top, bottom = np.where(rows)[0][[0, -1]]
    
    return np.array([left, top, right, bottom])

In [28]:
import cv2
import numpy as np
import requests
from io import BytesIO
from tqdm.notebook import tqdm

def get_image_array(url):
    response = requests.get(url)
    img_bytes = BytesIO(response.content)
    img_cv2 = cv2.imdecode(np.frombuffer(img_bytes.read(), np.uint8), -1)
    img_np = np.asarray(img_cv2)
    return img_np

In [29]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [47]:
sam_boxes = []
sum_iou = 0
n = 0
progress = tqdm(image_grounding.iterrows(), total=len(image_grounding))
for _, row in progress:
    url = row['image']
    input_box = np.array([row['pred_left'], row['pred_top'], row['pred_right'], row['pred_bottom']])
    gt_box = np.array([row['left'], row['top'], row['right'], row['bottom']])

    image = get_image_array(url)
    # image = np.asarray(Image.open(os.path.join('imgs', image_path)))
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = cv2.imread(os.path.join('imgs', image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


    predictor.set_image(image)

    masks, _, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :],
        multimask_output=False,
    )

    predicted_box = get_bounding_box(masks[0])
    iou = get_iou(
        {'x1': gt_box[0], 'y1': gt_box[1], 'x2': gt_box[2], 'y2': gt_box[3]},
        {'x1': predicted_box[0], 'y1': predicted_box[1], 'x2': predicted_box[2], 'y2': predicted_box[3]}
    )
    sam_boxes.append(predicted_box)
    sum_iou += iou
    n += 1
    avg_iou = sum_iou / n
    progress.set_description(f'IoU: {round(avg_iou, 2) * 100}')

  0%|          | 0/600 [00:00<?, ?it/s]

In [48]:
image_grounding['pred_left'] = [x[0] for x in bounding_boxes]
image_grounding['pred_top'] = [x[1] for x in bounding_boxes]
image_grounding['pred_right'] = [x[2] for x in bounding_boxes]
image_grounding['pred_bottom'] = [x[3] for x in bounding_boxes]

In [49]:
image_grounding['iou'] = image_grounding.apply(
    lambda x: get_iou(
        {'x1': x['left'], 'y1': x['top'], 'x2': x['right'], 'y2': x['bottom']},
        {'x1': x['pred_left'], 'y1': x['pred_top'], 'x2': x['pred_right'], 'y2': x['pred_bottom']}
    ),
    axis=1
)
print('Mean IoU:', image_grounding['iou'].mean())

Mean IoU: 0.3756524882406745


In [50]:
image_grounding.to_csv('ofa_sam_corrected.csv', index=False)